# The Next Hokage

### Helper Function

In [1]:
import cv2
# ffpyplayer for playing audio
from ffpyplayer.player import MediaPlayer

def PlayVideo(video_path):
    """Play video from specified video path."""
    video = cv2.VideoCapture(video_path)
    player = MediaPlayer(video_path)
    while True:
        grabbed, frame = video.read()
        audio_frame, val = player.get_frame()
        if not grabbed:
            # print("End of video")
            break
        if cv2.waitKey(28) & 0xFF == ord("q"):
            break
        cv2.imshow("Video", frame)
        if val != 'eof' and audio_frame is not None:
            # audio
            img, t = audio_frame
    video.release()
    cv2.destroyAllWindows()

In [2]:
jutsu_list = []
    
class Jutsu:
    """
    A base class for creating Jutsu.
    
    Attributes:
        name (str): Name of Jutsu technique
        success (list): Jutsu technique pattern.
        damage (int): Amount of damage.
        video_path (Path): Path to the Jutsu animation.
        hand_signs_detected (list): List of successful Jutsu signs.
        detected (bool): Flag to enable video playing.
    """
    
    def __init__(self, name, success, damage, video_path):
        """
        Initializes the Jutsu class.

        Args:
            name (str): Name of Jutsu technique
            success (list): Jutsu technique pattern.
            damage (int): Amount of damage.
            video_path (Path): Path to the Jutsu animation.
        """
        jutsu_list.append(self)
        self.name = name
        self.success = success
        self.damage = damage
        self.video_path = video_path
        self.hand_signs_detected = []
        self.detected = False
    
    def check_pattern(self, hand_sign):
        """Validate Jutsu pattern."""
        if len(self.hand_signs_detected) == len(self.success):
            self.detected = True
        elif hand_sign == self.success[len(self.hand_signs_detected)]:
            self.hand_signs_detected.append(hand_sign)
            print(f'{hand_sign} detected.')
    
    def reset(self):
        """Reset lists of successful Jutsu signs."""
        for jutsu in jutsu_list:
            jutsu.hand_signs_detected = []
            jutsu.detected = False

### Main Code

Create two Justu as example for the game.

In [3]:
import os 

name = 'Summoning Jutsu'
success = ['boar', 'dog', 'bird', 'monkey', 'ram']
damage = 200
video_path = os.getcwd() + "\\animation\\summoning_jutsu_vid.mp4"
summoningJutsu = Jutsu(name, success, damage, video_path)

In [4]:
name = 'Fireball Jutsu'
success = ['horse', 'tiger', 'snake', 'ram', 'monkey', 'boar', 'horse', 'tiger']
damage = 300
video_path = os.getcwd() + "\\animation\\fireball_jutsu_vid.mp4"
fireballJutsu = Jutsu(name, success, damage, video_path)

In [5]:
name = 'Chidori Jutsu'
success = ['ox', 'hare', 'monkey']
damage = 100
video_path = os.getcwd() + "\\animation\\chidori_jutsu_vid.mp4"
chidoriJutsu = Jutsu(name, success, damage, video_path)

### Start the game!

Instruction: Player has to display specific Jutsu technique pattern to deal damage. The Jutsu technique is described on the start of the game. Once the Jutsu technique is accomplised, an animation will be displayed as well as the damage done. 

In [6]:
from ultralytics import YOLO
import cv2


model = YOLO('best.pt')

# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# threshold
thres = 0.4

# display instructions
print('Welcome to The Next Hokage!')
print('Jutsu Techniques:')
for jutsu in jutsu_list:
    print(f'{jutsu.name}: {", ".join(jutsu.success)}')
print()

while True:
    success, img = cap.read()
    
    # vertically flip image
    img = cv2.flip(img, 1)
    
    results = model(img, stream=True, verbose=False) # conf=thres

    for result in results:
        img = result.plot()
        if len(result.boxes.cls) == 1:
            hand_sign = result.names[int(result.boxes.cls)]
        else:
            hand_sign = None

    cv2.imshow('Webcam', img)

    if hand_sign is not None:
        for jutsu in jutsu_list:
            jutsu.check_pattern(hand_sign)
            if jutsu.detected:
                PlayVideo(jutsu.video_path)
                print(f'Player deployed {jutsu.name} and dealed {jutsu.damage} damage.')
                jutsu.reset()
                
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Player dealed 300 dmg
